# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas
reviews = pandas.read_csv('/content/drive/MyDrive/IMDB Dataset.csv', encoding='utf-8',nrows=1000)


reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [ ]:
#Split the data into train and split

from sklearn.model_selection import train_test_split

In [ ]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
import pandas as pd
X= reviews.review
y = reviews.sentiment
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

train_reviews = pd.DataFrame(X_train)
test_reviews= pd.DataFrame(X_test)
train_sentiment  = pd.DataFrame(y_train)
test_sentiment = pd.DataFrame(y_test)

train_sentiment[train_sentiment == 'positive'] = 1
train_sentiment[train_sentiment == 'negative'] = 0
test_sentiment[test_sentiment == 'positive'] = 1
test_sentiment[test_sentiment == 'negative'] = 0
train_sentiment


,sentiment
132,0
15,0
505,1
950,1
444,0
...,...
41,1
788,0
864,1
170,0


In [ ]:
train_reviews.head()

,review
132,"""The Danish Bladerunner"" is boldly stated on t..."
15,"Kind of drawn in by the erotic scenes, only to..."
505,I know that you've already entered this in fil...
950,What a good movie! At last a picture revealing...
444,"First of all, the reason I'm giving this film ..."


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [ ]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    token_seqs = [[word.lower_ for word in encoder(text_seq)] for text_seq in text_seqs]
    return token_seqs


train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
132,"""The Danish Bladerunner"" is boldly stated on t...","["", the, danish, bladerunner, "", is, boldly, s..."
15,"Kind of drawn in by the erotic scenes, only to...","[kind, of, drawn, in, by, the, erotic, scenes,..."
505,I know that you've already entered this in fil...,"[i, know, that, you, 've, already, entered, th..."
950,What a good movie! At last a picture revealing...,"[what, a, good, movie, !, at, last, a, picture..."
444,"First of all, the reason I'm giving this film ...","[first, of, all, ,, the, reason, i, 'm, giving..."
541,An idiotic dentist finds out that his wife has...,"[an, idiotic, dentist, finds, out, that, his, ..."
694,Of the elements that make this the best at thi...,"[of, the, elements, that, make, this, the, bes..."
592,As is often the case when you attempt to take ...,"[as, is, often, the, case, when, you, attempt,..."
598,I have watched Love Jones over thirty times. I...,"[i, have, watched, love, jones, over, thirty, ..."
137,C'mon guys some previous reviewers have nearly...,"[c'mon, guys, some, previous, reviewers, have,..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [ ]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1
    

    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon = [token for token, count in token_counts.items() if count >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'"': 2, 'the': 3, 'danish': 4, 'bladerunner': 5, 'is': 6, 'boldly': 7, 'stated': 8, 'on': 9, 'box': 10, '.': 11, 'are': 12, 'you': 13, 'kidding': 14, 'me': 15, '?': 16, '!': 17, 'this': 18, 'film': 19, 'a': 20, 'complete': 21, 'drag': 22, 'when': 23, 'i': 24, "'m": 25, 'thirsty': 26, 'and': 27, 'go': 28, 'for': 29, 'soda': 30, 'in': 31, 'kitchen': 32, ',': 33, 'usually': 34, 'pause': 35, 'vcr': 36, 'so': 37, 'wo': 38, "n't": 39, 'miss': 40, 'anything': 41, 'not': 42, 'time': 43, 'actually': 44, 'found': 45, 'myself': 46, 'looking': 47, 'long': 48, 'hard': 49, 'fridge': 50, 'just': 51, 'would': 52, 'have': 53, 'to': 54, 'back': 55, 'why': 56, 'hell': 57, 'there': 58, 'one': 59, 'sciencefiction': 60, '-': 61, 'scriptwriter': 62, 'out': 63, 'who': 64, 'has': 65, 'vaguest': 66, 'clue': 67, 'about': 68, 'how': 69, 'computers': 70, 'work': 71, 'it': 72, "'s": 73, 'mindboggling': 74, 'of': 75, 'premises': 76, 'that': 77, 'our': 78, 'hero': 79, '(': 80, 'hacker': 81, ')': 82, 'little': 83, 'c

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [ ]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq]  
                                                                     for token_seq in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
132,"["", the, danish, bladerunner, "", is, boldly, s...","[2, 3, 4, 5, 2, 6, 7, 8, 9, 3, 10, 11, 12, 13,..."
15,"[kind, of, drawn, in, by, the, erotic, scenes,...","[118, 75, 119, 31, 120, 3, 121, 122, 33, 123, ..."
505,"[i, know, that, you, 've, already, entered, th...","[24, 199, 77, 13, 130, 200, 201, 18, 31, 19, 2..."
950,"[what, a, good, movie, !, at, last, a, picture...","[137, 20, 263, 96, 17, 203, 264, 20, 265, 266,..."
444,"[first, of, all, ,, the, reason, i, 'm, giving...","[325, 75, 142, 33, 3, 326, 24, 25, 327, 18, 19..."
541,"[an, idiotic, dentist, finds, out, that, his, ...","[393, 394, 395, 396, 63, 77, 335, 397, 65, 191..."
694,"[of, the, elements, that, make, this, the, bes...","[75, 3, 421, 77, 422, 18, 3, 210, 203, 18, 300..."
592,"[as, is, often, the, case, when, you, attempt,...","[225, 6, 218, 3, 475, 23, 13, 347, 54, 476, 20..."
598,"[i, have, watched, love, jones, over, thirty, ...","[24, 53, 654, 302, 655, 656, 657, 658, 11, 72,..."
137,"[c'mon, guys, some, previous, reviewers, have,...","[706, 707, 647, 708, 709, 53, 710, 667, 20, 71..."


In [ ]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = numpy.array([numpy.bincount(numpy.array(idx_seq), minlength=matrix_length) 
                            for idx_seq in idx_seqs])
    return bow_seqs
    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pandas.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 4 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 [0 0 4 ... 1 1 1]]
SHAPE: (670, 16538) 



,Word,Count
0,,0
1,<UNK>,0
2,"""",4
3,the,13
4,danish,1
...,...,...
16533,ers,0
16534,foxx.<br,0
16535,see.<br,0
16536,sustained,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len =max([len(idx_seq) for idx_seq in idx_seqs])
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len) 
    return padded_idxs
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[   0    0    0 ...  115  116  117]
 [   0    0    0 ...   31  197  198]
 [   0    0    0 ...  260  261  262]
 ...
 [   0    0    0 ...   31 1469   11]
 [   0    0    0 ...   31   72   11]
 [   0    0    0 ...   20  328   11]]
SHAPE: (670, 1446) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [ ]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))
    
    # Layer 2
    embedding_layer = Embedding(input_dim=n_input_nodes,
                                output_dim=n_embedding_nodes,
                                mask_zero=True)(input_layer)

    # Layer 3
    gru_layer = GRU(units=n_hidden_nodes)(embedding_layer)
    # Layer 4
    output_layer = Dense(units=1)(gru_layer)
    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(loss="mean_squared_error", optimizer='adam')
    return model


In [ ]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [ ]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
import numpy as np

y_tensor = tf.convert_to_tensor(train_sentiment[['sentiment']].values , dtype=tf.int64)
#train_reviews['review'] = tf.convert_to_tensor(train_reviews['review'])
#train_padded_idxs = tf.convert_to_tensor(train_padded_idxs)
rnn_model.fit(x=train_padded_idxs, y=y_tensor, batch_size=20, epochs=3)


Epoch 1/3
34/34 [==============================] - 420s 12s/step - loss: 0.2753
Epoch 2/3
34/34 [==============================] - 434s 13s/step - loss: 0.1681
Epoch 3/3
34/34 [==============================] - 407s 12s/step - loss: 0.0660


###  <font color='#6629b2'>Prediction</font>

In [ ]:
'''Put test reviews in padded matrix just how we did for train_reviews'''

test_reviews['Tokenized_Review'] = text_to_tokens(test_reviews['review'])
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'],
                                             lexicon=lexicon)
test_padded_idxs = idx_seqs_to_bows(test_reviews['Review_Idxs'], 
                                    matrix_length=len(lexicon) + 1) #add one to length for padding)


print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[ 0  2  0 ...  0  0  0]
 [ 0 11  8 ...  0  0  0]
 [ 0  9 10 ...  0  0  0]
 ...
 [ 0 34  0 ...  0  0  0]
 [ 0  6  0 ...  0  0  0]
 [ 0  4  0 ...  0  0  0]]
SHAPE: (330, 16538) 



In [ ]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [ ]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
y_test = tf.convert_to_tensor(test_sentiment[['sentiment']].values , dtype=tf.int64)
r2 = r2_score(y_true=y_test, y_pred=test_reviews['RNN_Pred_Rating'])
print("COEFFICIENT OF DETERMINATION (R2): {:3f}".format(r2))

COEFFICIENT OF DETERMINATION (R2): -0.964286


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [ ]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''
import gc 
gc.collect(generation=2)

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.